In [1]:
import os
import numpy as np
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.errors import HttpError
import pandas as pd
import json

scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]


def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    # This file needs to be registered and downloaded by yourself
    client_secrets_file = "client_secret_180985915353-vv0an1tjaqaacdcv7jhqjf8goe5p0769.apps.googleusercontent.com.json"
    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_local_server()

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    videoId = '8H99k8ormAo'
    request = youtube.commentThreads().list(
        part="snippet,replies",
        videoId=videoId,
        maxResults=100
    )
    response = request.execute()

    totalResults = int(response['pageInfo']['totalResults'])

    count = 0
    nextPageToken = ''
    comments = []
    first = True
    further = True
    while further:
        halt = False
        if first == False:
            print('..')
            try:
                response = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=videoId,
                    maxResults=100,
                    textFormat='plainText',
                    pageToken=nextPageToken
                ).execute()
                totalResults = int(response['pageInfo']['totalResults'])
            except HttpError as e:
                print("An HTTP error %d occurred:\n%s" % (e.resp.status, e.content))
                halt = True

        if halt == False:
            count += totalResults
            for item in response["items"]:
                # This is only a part of the data. 
                # You can choose what you need. You can print the data information you can get and crawl it as needed.
                comment = item["snippet"]["topLevelComment"]
                author = comment["snippet"]["authorDisplayName"]
                text = comment["snippet"]["textDisplay"]
                likeCount = comment["snippet"]['likeCount']
                publishtime = comment['snippet']['publishedAt']
                comments.append([author, publishtime, likeCount, text])
            if totalResults < 100:
                further = False
            else:
                further = True
                first = False
                try:
                    nextPageToken = response["nextPageToken"]
                except KeyError as e:
                    print("An KeyError error occurred: %s" % (e))
                    further = False
    print('get data count: ', str(count))

    ### write to csv file
    data = np.array(comments)
    df = pd.DataFrame(data, columns=['author', 'publishtime', 'likeCount', 'comment'])
    df.to_csv('中國雙十一帶貨主播太慘了，退貨率高達99%.csv', index=0, encoding='utf-8-sig')

    ### write to json file
    result = []
    for name, time, vote, comment in comments:
        temp = {}
        temp['author'] = name
        temp['publishtime'] = time
        temp['likeCount'] = vote
        temp['comment'] = comment
        result.append(temp)
    print('result: ', len(result))

    json_str = json.dumps(result, indent=4)
    with open('google_comments.json', 'w', encoding='utf-8-sig') as f:
        f.write(json_str)


if __name__ == "__main__":
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=180985915353-vv0an1tjaqaacdcv7jhqjf8goe5p0769.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=05zVFJPAouksHpfpfThYoPHvA9Atdv&access_type=offline
..
get data count:  124
result:  124
